In [ ]:
import time
import numpy as np
from PIL import Image

import chainer
from chainer import configuration
import chainer.links as L
import chainer.functions as F
from chainer import serializers

from lib.ev3 import EV3
from lib.vstream import VideoStream


touch_port = EV3.PORT_2
lmotor_port = EV3.PORT_B
rmotor_port = EV3.PORT_C


# Network definition
class MLP(chainer.Chain):

    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(300, 256)  # 300(20*15) -> 256 units
            self.l2 = L.Linear(256, 256)  # 256 units -> 256 units
            self.l3 = L.Linear(256, 1)      # 256 units -> 1

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


# Set up a neural network of trained model
predictor = MLP()

# Load the model
serializers.load_npz('ml_linetrace_model/mlp.model', predictor)

# Run VideoStream by setting image_size and fps
vs = VideoStream(resolution=(20, 15),
                 framerate=10,
                 colormode='binary').start()

ev3 = EV3()
ev3.enable_watchdog_task()
ev3.motor_config(lmotor_port, EV3.LARGE_MOTOR)
ev3.motor_config(rmotor_port, EV3.LARGE_MOTOR)
ev3.sensor_config(touch_port, EV3.TOUCH_SENSOR)

print("Push the touch sensor to start the linetracer")
while not ev3.touch_sensor_is_pressed(touch_port):
    pass

# Confirm the touch sensor is released.
while ev3.touch_sensor_is_pressed(touch_port):
    pass

# Enable evaluation mode for faster inference.
with configuration.using_config('train', False), chainer.using_config('enable_backprop', False):
    while True:
        # Break this loop when the touch sensor was pressed.
        if ev3.touch_sensor_is_pressed(touch_port):
            break
        im = vs.read()  # Get a current image in PIL format.
        im = np.asarray(im, dtype=np.float32)  # Convert to numpy array.
        x = im / 255.  # Normalization
        x = x.reshape(1, 300)  # (20, 15) -> (1, 300)
        y = predictor(x)  # Predict steer value from x.
        steer = y.data[0, 0]
        print("predicted steer = {}".format(steer))
        ev3.motor_steer(lmotor_port, rmotor_port, 10, int(steer))

vs.stop()
ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
ev3.close()